<a href="https://colab.research.google.com/github/piyushghante/streamlit-example/blob/master/SIH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install mysql-connector-python


In [2]:
import mysql.connector
import os

def retrieve_documents_from_db():
    # Set up MySQL connection
    db_connection = mysql.connector.connect(
        host="sql12.freemysqlhosting.net",
        user="sql12647419",
        password="vKb8GgZwFe",
        database="sql12647419"
    )

    # Initialize MySQL cursor
    cursor = db_connection.cursor()

    # Retrieve documents from the database
    select_query = "SELECT id, text FROM documents;"
    cursor.execute(select_query)

    # Create a directory to store the documents
    output_dir = "data"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Loop through the retrieved documents
    for row in cursor.fetchall():
        doc_id, document_text = row

        # Create a file path using the document ID as the filename
        file_name = f"{doc_id}.txt"
        file_path = os.path.join(output_dir, file_name)

        # Save the document text to the file
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(document_text)

    # Close the cursor and connection
    cursor.close()
    db_connection.close()

# Example usage:
retrieve_documents_from_db()


In [ ]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,inference]


In [7]:
from haystack.telemetry import tutorial_running

tutorial_running(1)


In [8]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)


In [ ]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)


In [10]:
doc_dir = "data"

In [ ]:
import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline

files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline = TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(file_paths=files_to_index)


In [12]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)


In [ ]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)


In [19]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)


In [79]:
prediction = pipe.run(
    query="Notice of reopening", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 3}}
)


Inferencing Samples: 100%|██████████| 1/1 [00:15<00:00, 15.10s/ Batches]


In [80]:
from pprint import pprint

pprint(prediction)


{'answers': [<Answer {'answer': 'the owner, agent or manager shall, not less than thirty\ndays before resumption of mining operations', 'type': 'extractive', 'score': 0.06911326944828033, 'context': 'iod exceeding sixty days, the owner, agent or manager shall, not less than thirty\ndays before resumption of mining operations, give to the Chief Inspe', 'offsets_in_document': [{'start': 151, 'end': 250}], 'offsets_in_context': [{'start': 26, 'end': 125}], 'document_ids': ['e2378d1b3a22617fc407dc85d2fb60ce'], 'meta': {'_split_id': 8}}>,
             <Answer {'answer': 'night', 'type': 'extractive', 'score': 0.03561824560165405, 'context': 'e mine or any part thereof or any output therefrom at any time by day or\nnight:\nProvided that, where in the opinion of the Chief Inspector or of an In', 'offsets_in_document': [{'start': 614, 'end': 619}], 'offsets_in_context': [{'start': 73, 'end': 78}], 'document_ids': ['cd26f3254fb6b546d5f55b8edafe7431'], 'meta': {'_split_id': 13}}>,
             <

In [81]:
from haystack.utils import print_answers

print_answers(prediction, details="minimum")  ## Choose from `minimum`, `medium`, and `all`


'Query: Notice of reopening'
'Answers:'
[   {   'answer': 'the owner, agent or manager shall, not less than thirty\n'
                  'days before resumption of mining operations',
        'context': 'iod exceeding sixty days, the owner, agent or manager '
                   'shall, not less than thirty\n'
                   'days before resumption of mining operations, give to the '
                   'Chief Inspe'},
    {   'answer': 'night',
        'context': 'e mine or any part thereof or any output therefrom at any '
                   'time by day or\n'
                   'night:\n'
                   'Provided that, where in the opinion of the Chief Inspector '
                   'or of an In'},
    {   'answer': 'prescribed',
        'context': ' form and containing such particulars relating to the '
                   'mine, as may be\n'
                   'prescribed.\n'
                   '(2) Any notice given under sub-section (1) shall be so '
                   'given 

In [82]:
from haystack.pipelines import ExtractiveQAPipeline

# ... (previous code)

# Increase the top_k for both Retriever and Reader components
prediction = pipe.run(
    query="What is the process for the Central Government to recover the cost of measures undertaken in relation to a coal mine? ",
    params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

# Extract the answers from the prediction
answers = prediction["answers"]

# Combine the answers into a single response
combined_answer = "\n".join(answer.answer for answer in answers)

# Print the combined answer
print("Combined Answer:")
print(combined_answer)


Inferencing Samples: 100%|██████████| 1/1 [00:12<00:00, 12.75s/ Batches]

Combined Answer:
sub-section (1) or
sub-section (2) in relation to a coal mine is justified, recover such cost from the owner, agent or manager
after due appropriation made by Parliament by law
Coal Mines Provident Fund Scheme
the Central Government shall constitute a Tribunal
ancillary to the getting, dressing or preparation for sale of coal obtained as a result of such
operations
